In [1]:

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
model_path = "conf1-lstm-model.h5"
model = load_model(model_path)

In [3]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

In [91]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

In [92]:
from jaad_data import JAAD

In [93]:
imdb = JAAD(data_path = jaad_path)

In [94]:
vid = "video_0071"

In [95]:
anno = imdb._get_annotations(vid)

In [96]:
l = [k for k,v in anno['ped_annotations']['0_71_386b'].items()]

In [97]:
len(anno['ped_annotations']['0_71_386b']['frames'])

297

In [98]:
len(anno['ped_annotations']['0_71_386b']['bbox'])

297

In [99]:
l

['behavior', 'bbox', 'old_id', 'frames', 'occlusion']

In [100]:
anno['width']

1920

In [101]:
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]
print(ped_b)
for i, p in enumerate(ped_b):
    if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
#print(bb_cross)

['0_71_386b', '0_71_384b']


In [102]:
len(bb_cross[ped_b[0]])

297

In [127]:
lstm_seq =  bb_cross[ped_b[0]]

In [128]:
lstm_seq

[[1248.0, 620.0, 1299.0, 747.0],
 [1253.0, 619.0, 1304.0, 748.0],
 [1258.0, 618.0, 1308.0, 749.0],
 [1263.0, 617.0, 1313.0, 750.0],
 [1268.0, 616.0, 1318.0, 751.0],
 [1274.0, 615.0, 1323.0, 753.0],
 [1279.0, 614.0, 1328.0, 754.0],
 [1284.0, 613.0, 1333.0, 755.0],
 [1289.0, 612.0, 1337.0, 756.0],
 [1294.0, 611.0, 1342.0, 757.0],
 [1299.0, 609.0, 1348.0, 757.0],
 [1303.0, 608.0, 1352.0, 758.0],
 [1308.0, 606.0, 1358.0, 758.0],
 [1313.0, 604.0, 1363.0, 758.0],
 [1317.0, 602.0, 1368.0, 758.0],
 [1322.0, 601.0, 1374.0, 759.0],
 [1327.0, 599.0, 1379.0, 759.0],
 [1331.0, 597.0, 1384.0, 759.0],
 [1336.0, 595.0, 1389.0, 759.0],
 [1341.0, 594.0, 1395.0, 760.0],
 [1345.0, 592.0, 1400.0, 760.0],
 [1350.0, 590.0, 1405.0, 760.0],
 [1355.0, 588.0, 1411.0, 760.0],
 [1359.0, 587.0, 1415.0, 761.0],
 [1364.0, 585.0, 1421.0, 761.0],
 [1368.0, 585.0, 1426.0, 762.0],
 [1373.0, 585.0, 1431.0, 763.0],
 [1377.0, 585.0, 1436.0, 764.0],
 [1382.0, 586.0, 1442.0, 765.0],
 [1386.0, 586.0, 1447.0, 766.0],
 [1391.0, 

In [104]:
frames = len(anno['ped_annotations']['0_71_386b']['frames'])

In [105]:
frames

297

In [106]:
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

In [107]:
df.loc[0:0, :] = np.array([0] * 4) 

In [108]:
df 

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,1253.0,619.0,1304.0,748.0
2,1258.0,618.0,1308.0,749.0


In [109]:
pickle_in = open("min-max-scaler.pkl","rb")
scaler = pickle.load(pickle_in)
print(type(scaler))
print(scaler.data_max_)
print(scaler.data_min_)

<class 'sklearn.preprocessing.data.MinMaxScaler'>
[1900.  953. 1919. 1079.]
[0. 0. 0. 0.]


In [110]:
global batch

In [129]:
lstm_seq = scaler.transform(lstm_seq)

In [130]:
lstm_seq

array([[0.66, 0.65, 0.68, 0.69],
       [0.66, 0.65, 0.68, 0.69],
       [0.66, 0.65, 0.68, 0.69],
       ...,
       [0.  , 0.5 , 0.05, 1.  ],
       [0.  , 0.52, 0.05, 1.  ],
       [0.  , 0.54, 0.04, 1.  ]])

In [122]:
batch = scaler.transform(df)

In [123]:
batch

array([[0.  , 0.  , 0.  , 0.  ],
       [0.66, 0.65, 0.68, 0.69],
       [0.66, 0.65, 0.68, 0.69]])

In [124]:
def play():
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print( length )
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0
    pred = True

    # Capture frame-by-frame
    ret = True
    while(ret):
        ret, origimg = cap.read()
        lstm_seq
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False and i < frames ):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            batch =  np.delete(batch, 0, axis = 0)
            batch =  np.append(batch, y_hat, axis = 0)
            print(batch)
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
            
            
            
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)

            if pred:
                xmin = int(inv_yhat[0] / 2)
                ymin = int(inv_yhat[1] / 2)
                xmax = int(inv_yhat[2] / 2)
                ymax = int(inv_yhat[3] / 2)
            else:
                xmin = int(bb_cross[ped_b[0]][i][0] / 2)
                ymin = int(bb_cross[ped_b[0]][i][1] / 2)
                xmax = int(bb_cross[ped_b[0]][i][2] / 2)
                ymax = int(bb_cross[ped_b[0]][i][3] / 2)


            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)

            # Draw the predicted boxes in blue
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [125]:
play()

330
[[0.66 0.65 0.68 0.69]]
[[0.66 0.65 0.68 0.69]
 [0.66 0.65 0.68 0.69]
 [0.66 0.65 0.68 0.69]]
[1258.95  619.   1314.13  749.25]
(629, 309)
(657, 374)
[[0.66 0.65 0.69 0.69]]
[[0.66 0.65 0.68 0.69]
 [0.66 0.65 0.68 0.69]
 [0.66 0.65 0.69 0.69]]
[1258.    620.41 1322.99  749.7 ]
(628, 310)
(661, 374)
[[0.66 0.65 0.69 0.7 ]]
[[0.66 0.65 0.68 0.69]
 [0.66 0.65 0.69 0.69]
 [0.66 0.65 0.69 0.7 ]]
[1255.93  622.21 1333.55  750.4 ]
(627, 311)
(666, 375)
[[0.66 0.65 0.7  0.7 ]]
[[0.66 0.65 0.69 0.69]
 [0.66 0.65 0.69 0.7 ]
 [0.66 0.65 0.7  0.7 ]]
[1253.64  624.17 1344.95  751.42]
(626, 312)
(672, 375)
[[0.66 0.66 0.71 0.7 ]]
[[0.66 0.65 0.69 0.7 ]
 [0.66 0.65 0.7  0.7 ]
 [0.66 0.66 0.71 0.7 ]]
[1251.79  626.12 1356.49  752.8 ]
(625, 313)
(678, 376)
[[0.66 0.66 0.71 0.7 ]]
[[0.66 0.65 0.7  0.7 ]
 [0.66 0.66 0.71 0.7 ]
 [0.66 0.66 0.71 0.7 ]]
[1250.86  627.89 1367.64  754.52]
(625, 313)
(683, 377)
[[0.66 0.66 0.72 0.7 ]]
[[0.66 0.66 0.71 0.7 ]
 [0.66 0.66 0.71 0.7 ]
 [0.66 0.66 0.72 0.7 ]]
[1

[[1.63 0.47 1.71 0.64]]
[[1.53 0.49 1.64 0.69]
 [1.58 0.48 1.68 0.67]
 [1.63 0.47 1.71 0.64]]
[3105.92  446.18 3281.23  693.7 ]
(1552, 223)
(1640, 346)
[[1.68 0.46 1.74 0.62]]
[[1.58 0.48 1.68 0.67]
 [1.63 0.47 1.71 0.64]
 [1.68 0.46 1.74 0.62]]
[3191.9   439.66 3341.74  666.87]
(1595, 219)
(1670, 333)
[[1.72 0.46 1.77 0.59]]
[[1.63 0.47 1.71 0.64]
 [1.68 0.46 1.74 0.62]
 [1.72 0.46 1.77 0.59]]
[3265.97  434.01 3396.72  640.3 ]
(1632, 217)
(1698, 320)
[[1.75 0.45 1.8  0.57]]
[[1.68 0.46 1.74 0.62]
 [1.72 0.46 1.77 0.59]
 [1.75 0.45 1.8  0.57]]
[3327.35  428.99 3445.98  614.65]
(1663, 214)
(1722, 307)
[[1.78 0.45 1.82 0.55]]
[[1.72 0.46 1.77 0.59]
 [1.75 0.45 1.8  0.57]
 [1.78 0.45 1.82 0.55]]
[3376.49  424.44 3489.36  590.44]
(1688, 212)
(1744, 295)
[[1.8  0.44 1.84 0.53]]
[[1.75 0.45 1.8  0.57]
 [1.78 0.45 1.82 0.55]
 [1.8  0.44 1.84 0.53]]
[3414.83  420.21 3526.75  567.96]
(1707, 210)
(1763, 283)
[[1.81 0.44 1.85 0.51]]
[[1.78 0.45 1.82 0.55]
 [1.8  0.44 1.84 0.53]
 [1.81 0.44 1.85 0

[[ 2.22  0.42  2.18 -0.08]]
[[ 2.21  0.41  2.18 -0.07]
 [ 2.22  0.42  2.18 -0.08]
 [ 2.22  0.42  2.18 -0.08]]
[4220.26  396.47 4191.59  -86.33]
(2110, 198)
(2095, -43)
[[ 2.22  0.42  2.19 -0.08]]
[[ 2.22  0.42  2.18 -0.08]
 [ 2.22  0.42  2.18 -0.08]
 [ 2.22  0.42  2.19 -0.08]]
[4226.71  397.08 4197.12  -89.78]
(2113, 198)
(2098, -44)
[[ 2.23  0.42  2.19 -0.09]]
[[ 2.22  0.42  2.18 -0.08]
 [ 2.22  0.42  2.19 -0.08]
 [ 2.23  0.42  2.19 -0.09]]
[4232.67  397.65 4202.23  -92.95]
(2116, 198)
(2101, -46)
[[ 2.23  0.42  2.19 -0.09]]
[[ 2.22  0.42  2.19 -0.08]
 [ 2.23  0.42  2.19 -0.09]
 [ 2.23  0.42  2.19 -0.09]]
[4238.16  398.17 4206.93  -95.85]
(2119, 199)
(2103, -47)
[[ 2.23  0.42  2.19 -0.09]]
[[ 2.23  0.42  2.19 -0.09]
 [ 2.23  0.42  2.19 -0.09]
 [ 2.23  0.42  2.19 -0.09]]
[4243.22  398.65 4211.26  -98.52]
(2121, 199)
(2105, -49)
[[ 2.24  0.42  2.2  -0.09]]
[[ 2.23  0.42  2.19 -0.09]
 [ 2.23  0.42  2.19 -0.09]
 [ 2.24  0.42  2.2  -0.09]]
[4247.87  399.1  4215.24 -100.96]
(2123, 199)
(210

[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.88  403.27 4253.02 -122.93]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.88  403.27 4253.02 -122.94]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.88  403.27 4253.02 -122.94]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.89  403.27 4253.02 -122.94]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.89  403.27 4253.03 -122.94]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.89  403.27 4253.03 -122.94]
(2145, 201)
(212

[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(212

[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(2126, -61)
[[ 2.26  0.42  2.22 -0.11]]
[[ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]
 [ 2.26  0.42  2.22 -0.11]]
[4291.91  403.27 4253.04 -122.95]
(2145, 201)
(212